# Homework 4 - Hard Sphere Phase Behavior

For this homework assignment, we will look at the structural ordering of hard spheres within different phase boundaries. **Simulate 5 systems of hard spheres at different volume fractions, $\mathbf{\phi = 0.4, 0.5, 0.6, 0.7, \text{and} \: 0.74}$.** 

**For each simulation, note the phase of the system and calculate the RDF (see `HW4-Analysis.ipynb`). Please submit your responses to the questions and RDF plots in addition to one plot of all RDFs with comparisons in a PDF file along with the rest of your homework solutions and a PDF version of this notebook to Canvas.**

As always, we begin by importing our needed packages.

In [ ]:
import hoomd
import gsd.hoomd
import numpy as np
import itertools

This time around, I will let you set up the simulation yourself with a little outline to guide you. As always, we must first create our simulation object.

In [1]:
#choose your device and create the simulation object and integrator

Next, initial the neighborlist and create a Lennard-Jones force. To model psuedo-hard-sphere behavior, set the cutoff of the LJ potential to be $2^{1/6}\sigma$. 

In [2]:
#initialize your neighborlist and create a lennard-jones force. Set r_cut to 2^(1/6)

Now it's time to set up the integration methods. For these simulations, we want an NVT integration method with a Nosé-Hoover thermostat and $kT = 1.0$. Since we're doing NVT integration, we also have to assign non-zero velocities to the particles in our system. If you don't remember how to set this up, check the HOOMD documentation!

In [ ]:
#set up the integration methods

Next up is to create the compute object to compute our thermodynamic properties.

In [3]:
#compute thermodynamic properties

And the last thing you're going to do for now is set up your table logger. Note: Only set up the table logger now, we don't want to set up our GSD writer until later!

In [ ]:
#create table logger

Now that you've set up the main parts of the simulation, I'm going to add something new! As we discussed in class, hard spheres undergo phase transitions without the influence of temperature. This means that we can simulate the different phases of a hard sphere system by simply changing the volume fraction within the system! 

One of the easiest ways to change the volume fraction of the system in an MD simulation is by changing the size of the box, which is what we are going to set up here. The way HOOMD scales the simulation box is by using a **Variant** to interpolate between the initial and final box dimensions. We are going to use a **Ramp** variant to linearly scale the box. To do this we must first create our variant object, which is going to be a ramp.

In [ ]:
t_ramp = 1000
ramp = hoomd.variant.Ramp(A=0, B=1, t_start=sim.timestep, t_ramp=t_ramp)

The ramp we just created is going to start at the current timestep (`sim.timestep`) and end 1000 timesteps later. Now that we've established the variant, we can get our initial box from the simulation state, and calculate our initial volume fraction. The equation for volume fraction can be defined as: $$\phi=\frac{V_{particles}}{V_{box}} = \frac{N_{particles}*\left(\frac{4}{3}\pi r^3\right)}{L_{box}^3}$$ 

with $r$ representing the radius of the particle.

In [ ]:
init_box = sim.state.box
phi = (sim.state.N_particles*(4/3)*np.pi*(0.5**2))/init_box.volume

Next, we want to create our final box and calculate its volume using our desired volume fraction.

In [ ]:
final_box = hoomd.Box.from_box(init_box)
new_phi = 0.1
final_box.volume = (sim.state.N_particles*(4/3)*np.pi*(0.5**2))/new_phi

And finally, we can create our `BoxResize` updater. In HOOMD, updaters do exactly what it sounds like, they update the simulation state. We can give this updater a periodic trigger so that it can gradually change the box volume and we can ensure that we don't cause any undesired forces in the system. 

In [ ]:
box_resize = hoomd.update.BoxResize(box1=init_box, box2=final_box, 
                                   variant=ramp, trigger=hoomd.trigger.Periodic(10))
sim.operations.updaters.append(box_resize)

Let's now run the simulation for `t_ramp` timesteps and then remove the `box_resize` object from our list of updaters.

In [ ]:
sim.run(t_ramp)
sim.operations.updaters.remove(box_resize)

Now that we've rescaled the box to achieve our desired volume fraction, we can set up our GSD writer just like normal to log our simulation trajectory.

In [ ]:
#create GSD writer

And finally we can run the simulation. Don't forget to flush your GSD writer afterward!

In [ ]:
#run the simulation!

To do the post-processing for this simulation, see the analysis notebook!